# Import & Functions

In [ ]:
import os
import sys
import time
import glob
import numpy as np
import pandas as pd

import nibabel as nib
import SimpleITK as sitk

from scipy import ndimage
from skimage import filters
import cc3d
import cv2

# Please check following link for installing segmentation method
# https://github.com/JoHof/lungmask
from lungmask import mask

In [ ]:
def mkdir(dirpath):
    """
    Create directory if no directory with the given path
    :param dirpath: directory path
    :return: nothing but create directory (if needed)
    """
    if not os.path.exists(dirpath):
        print("Creating directory at:", dirpath)
        os.makedirs(dirpath)

In [ ]:
def body_segmentation(img_arr):
    '''
    Segmentation of the body
    :param img_arr: 3D image array
    :return: return a binary array of the segmentation 
    '''
    # Step 2 - 3 : Threshold value
    # T1 : separate air and human tissue
    img_arr = img_arr.astype(int)
    T1 = filters.threshold_otsu(img_arr)
    img_arr = np.where(img_arr < T1, -1000, img_arr)

    # T3 : separate muscle and skeleton
    T3 = filters.threshold_otsu(img_arr[img_arr > T1])
    
    img_arr = np.where(img_arr > T3, 1000, img_arr)
    img_arr = np.where(img_arr > -1000, 1, img_arr)

    # Step 4 : Connecting component processing
    # Find human body without background
    connectivity = 6
    labels_out, N = cc3d.connected_components(img_arr, connectivity=connectivity, return_N=True) # free
    
    # Need to find the index which represents the body
    idx, counts = np.unique(labels_out, return_counts=True)
    idx_tissues = idx[np.argmax(counts[1:])+1] # The first element is exclude because it's the background
    tissues = np.where(labels_out != idx_tissues, 0, 1)

    # Step 5 : Region filling
    # Filling lungs region per slice
    res = []
    for i in range(tissues.shape[0]):
        s = tissues[i, :, :]
        s = np.uint8(s)
        im_floodfill = s.copy()

        # Notice the size needs to be 2 pixels than the image.
        h, w = s.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)

        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (0,0), 255);
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        # Combine the two images to get the foreground.
        im_out = s | im_floodfill_inv
        im_out = np.where(im_out==255, 1,im_out)
        res += [im_out]
    filled = np.array(res)    
    return filled

In [ ]:
def calculate_bounding_box(segmentation_img):
    '''
    Determine the bounding box origins & sizes
    :param segmentation_img: 3D simple itk image
    :return: origins & size ([origin_X,origin_Y,origin_Z,size_X,size_Y,size_Z])
    '''
    inside_value = 0
    outside_value = 1
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(segmentation_img)
    bounding_box = label_shape_filter.GetBoundingBox(outside_value)
    
    return bounding_box

In [ ]:
def resample_image(input_f, output_f, reference_size = [128,128,128], out_type='int16', interpolation = sitk.sitkLinear):
    """
    Resize 3D data to fit the same size as wanted
    
    :param input_f: nifti file (.nii.gz)
    :param output_f: output .nii.gz file
    :param resize_size: image dims wanted, by default [256,256,256]
    :param out_type: output image type
    :param interpolation: interpolation mode (default: sitk.sitkLinear)
    
    """
    # Open file and get reference image details
    original_CT = sitk.ReadImage(input_f)
    dimension = original_CT.GetDimension()
    reference_origin = original_CT.GetOrigin()
    reference_direction = np.identity(dimension).flatten()
    
    reference_physical_size = np.zeros(dimension)
    reference_physical_size[:] = [(sz-1)*spc if sz*spc>mx  else mx for sz,spc,mx in zip(original_CT.GetSize(), original_CT.GetSpacing(), reference_physical_size)]
    
    # Setup the new size and the new voxel spacing
    reference_spacing = [ phys_sz/(sz-1) for sz,phys_sz in zip(reference_size, reference_physical_size) ]

    # Change image information
    reference_image = sitk.Image(reference_size, original_CT.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(reference_spacing)
    reference_image.SetDirection(reference_direction)

    # Translation
    reference_center = np.array(reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize())/2.0))
    
    transform = sitk.AffineTransform(dimension)
    transform.SetMatrix(original_CT.GetDirection())

    # transform.SetTranslation(np.array(original_CT.GetOrigin()) - reference_origin)
  
    centering_transform = sitk.TranslationTransform(dimension)
    img_center = np.array(original_CT.TransformContinuousIndexToPhysicalPoint(np.array(original_CT.GetSize())/2.0))
    centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))
    centered_transform = sitk.Transform(transform)
    
    # Resample with all new information
    res = sitk.Resample(original_CT, reference_image, centered_transform, interpolation, -1000.0) # last argument : defaultPixelValue
    
    # Save image
    if out_type == 'int16':
        castFilter = sitk.CastImageFilter()
        castFilter.SetOutputPixelType(sitk.sitkInt16)
    elif out_type == 'uint8':
        castFilter = sitk.CastImageFilter()
        castFilter.SetOutputPixelType(sitk.sitkUInt8)
    else:
        print('Error')
    res = castFilter.Execute(res)
    sitk.WriteImage(res, output_f)

Suppose that image are already in nifti and with the right naming : LungCT_case_number

# Preprocessing

## Create directories

In [ ]:
# Init path
csv_dir      = './datasets/000_csv/'
original_dir = './datasets/001_original/'
lung_dir = os.path.join(original_dir, 'lung')
body_dir = os.path.join(original_dir, 'body')

bb_dir       = './datasets/002_bounding_box/'
final_dir    = './datasets/003_128x128x128/'

# Create directories
dirs = [csv_dir, lung_dir, body_dir, bb_dir, final_dir]
for d in dirs:
    mkdir(d)

In [ ]:
isTumor = True # If need to run inference with already available tumor 
for d in dirs[3:]:
    mkdir(os.path.join(d, 'tumor'))

## Prepare bounding boxes

In [ ]:
image_list = sorted(glob.glob(original_dir + '/*.nii.gz'))
lung_bb_values = pd.DataFrame(columns=['case','phase','origin_X','origin_Y','origin_Z','size_X','size_Y','size_Z'])
body_bb_values = pd.DataFrame(columns=['case','phase','origin_X','origin_Y','origin_Z','size_X','size_Y','size_Z'])

for img_file in image_list[:]:
    name  = os.path.basename(img_file)
    case  = name.split('_')[1]
    phase = name.split('_')[-1].split('.')[0]
    print('Start preprocessing :',name)
    
    # Read image
    input_img = sitk.ReadImage(img_file)
    input_arr = sitk.GetArrayFromImage(input_img)
    
    # Lung Segmentation ---------------------------------------------
    # Segmentation 
    segmentation = mask.apply(input_arr)
    segmentation[segmentation > 1] = 1 # Don't need to distenguish right/left lobes
    
    lung_img = sitk.GetImageFromArray(segmentation)
    lung_img.CopyInformation(input_img)

    # Save segmentation
    lung_file = os.path.join(original_dir, 'lung', name)
    print("Save lung to : {}".format(lung_file))
    sitk.WriteImage(lung_img, lung_file)
    
    # Bounding box 
    lung_bb = calculate_bounding_box(lung_img)

    # Save bounding box values
    new_row = {'case': case,
               'phase': phase,
               'origin_X': lung_bb[0],
               'origin_Y': lung_bb[1],
               'origin_Z': lung_bb[2],
               'size_X': lung_bb[3],
               'size_Y': lung_bb[4],
               'size_Z': lung_bb[5]}
    lung_bb_values = lung_bb_values.append(new_row, ignore_index=True)
    
    # Body segmentation ----------------------------------------
    # Segmentation
    body_arr = body_segmentation(input_arr)
    body_img = sitk.GetImageFromArray(body_arr)
    body_img.CopyInformation(input_img)
    
    # Save segmentation
    body_file = os.path.join(original_dir, 'body', name)
    print("Save body to : {}".format(body_file))
    sitk.WriteImage(body_img,body_file)
    
    # Bounding box
    body_bb = calculate_bounding_box(body_img)
    
    # Save bounding box values
    new_row = {'case': case,
               'phase': phase,
               'origin_X': body_bb[0],
               'origin_Y': body_bb[1],
               'origin_Z': body_bb[2],
               'size_X': body_bb[3],
               'size_Y': body_bb[4],
               'size_Z': body_bb[5]
              }
    body_bb_values = body_bb_values.append(new_row, ignore_index=True)

    print()
lung_bb_values.to_csv(csv_dir + 'lung_bb.csv', index=False)
body_bb_values.to_csv(csv_dir + 'body_bb.csv', index=False)

## Crop with bounding boxes values

In [ ]:
image_list = sorted(glob.glob(original_dir + '/*.nii.gz'))
lung_bb_values = pd.read_csv(csv_dir + 'lung_bb.csv')
body_bb_values = pd.read_csv(csv_dir + 'body_bb.csv')

final_bb_values = pd.DataFrame(columns=['case','origin_X','origin_Y','origin_Z','size_X','size_Y','size_Z'])
pred = ''
for img_file in image_list[:]:
    # Check subject case
    name = os.path.basename(img_file)
    case = name.split('_')[1]
    if pred == case:
        continue
    print('Choose appropriate values for case LungCT_{}'.format(case))
    img = nib.load(img_file)
    dims = img.shape
    # print(dims)
    # Get all phases of the case
    all_phases = [s for s in image_list if 'LungCT_{}_'.format(case) in s]
    
    # Get the lung bounding boxes and image sizes values
    lung_values = lung_bb_values[lung_bb_values.case == int(case)]
    body_values = body_bb_values[body_bb_values.case == int(case)]
    
    # Check each case for the correct bounding boxes which respects following conditions:
    # 1. We want a higher part under the diaphragm 
    # 2. We keep a small part above the lung
    # 3. We want all surface of the abdomen
    
    # 1. Under diaphragm
    origin_z = lung_values.origin_Z.min()-5
    if origin_z < 0:
        print("     - Check case: LungCT_{} for origin".format(case))
        print("             Old : {} ; New : {}".format(origin_z, 0) )
        origin_z = 0
        
    # 2. Above lung
    size_z = lung_values.size_Z.max()+10
    if size_z+origin_z > dims[2]:
        print("     - Check case: LungCT_{} for size".format(case))
        print("             Old : {} ; New : {}".format(size_z, dims[2]-origin_z) )
        size_z = dims[2]-origin_z
        
    # 3. Abdomen surface
    origin_y = body_values.origin_Y.min()-5
    if origin_y < 0:
        print("     - Check case: LungCT_{} for origin".format(case))
        print("             Old : {} ; New : {}".format(origin_y, 0) )
        origin_y = 0

    size_y = lung_values.size_Y.max() + (lung_values.origin_Y.min() - origin_y) +15
    if size_y+origin_y > dims[1]:
        print("     - Check case: LungCT_{} for size".format(case))
        print("             Old : {} ; New : {}".format(size_y, dims[1]-origin_y) )
        size_y = dims[1]-origin_y

    # Save final bounding box values
    origins = np.array([lung_values.origin_X.min(), origin_y, origin_z], dtype='int').tolist()
    sizes   = np.array([lung_values.size_X.max()  , size_y  , size_z  ], dtype='int').tolist()
    
    print('     - Origins and sizes :',case, origins, sizes)

    new_row = {'case': case,
               'origin_X': origins[0],
               'origin_Y': origins[1],
               'origin_Z': origins[2],
               'size_X': sizes[0],
               'size_Y': sizes[1],
               'size_Z': sizes[2]}
    final_bb_values = final_bb_values.append(new_row, ignore_index=True)
    
    # For all phase, crop with biggest BB
    print('     - Crop images of case: LungCT_{}'.format(case))
    for phase in all_phases :
        # Image
        img_name = os.path.basename(phase)
        
        ## Crop and save
        img = sitk.ReadImage(phase, sitk.sitkInt16)
        img_bb = sitk.RegionOfInterest(img, sizes, origins)
        sitk.WriteImage(img_bb, os.path.join(bb_dir, img_name))
    
        if isTumor:
            # Tumor
            tumor_file = os.path.join(original_dir, 'tumor', img_name)
            tumor = sitk.ReadImage(tumor_file, sitk.sitkUInt8)
            tumor_bb = sitk.RegionOfInterest(tumor, sizes, origins)
            sitk.WriteImage(tumor_bb, os.path.join(bb_dir, 'tumor', img_name))
    
    pred = case
    print()
final_bb_values.to_csv(csv_dir + 'final_bb.csv')

## To 128x128x128

In [ ]:
iso_list = sorted(glob.glob(bb_dir + '*.nii.gz'))
ref_size = [128,128,128]
for iso_file in iso_list[:] :
    name = os.path.basename(iso_file)
    case = name.split('_')[1]
    print('Resampling file :{}'.format(name))
    # Image resample
    resampled_file = os.path.join(final_dir,name)
    resample_image(iso_file, resampled_file, reference_size=ref_size, out_type='int16')
    
    # Tumor resample
    if isTumor:
        tumor_file = os.path.join(bb_dir,'tumor',name)
        resampled_file = os.path.join(final_dir,'tumor',name)
        resample_image(tumor_file, resampled_file, reference_size=ref_size, out_type='uint8', interpolation=sitk.sitkNearestNeighbor)
    print()